In [ ]:
# All imports and setup
import pandas as pd
from sqlalchemy import create_engine, text
from tqdm.notebook import tqdm
tqdm.pandas()


# Add project root to path 
import sys
from pathlib import Path
sys.path.append(str(Path("../").resolve()))



In [ ]:
import src.preprocessing_utils as prep
from src.db_utils import get_db_credentials

In [ ]:
# Connect to PostgreSQL via SQLAlchemy
creds = get_db_credentials()
conn_str = f"postgresql+psycopg2://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['db_name']}"
engine = create_engine(conn_str)

# Load tickets (skip demo for pipeline demo purposes)
query = "SELECT ticket_id, subject, body FROM tickets WHERE demo_flag = FALSE;"
tickets_df = pd.read_sql(query, engine)

tickets_df.head()

In [ ]:
#  Preprocessing serial : Detect language and clean text
def preprocess_ticket_row(row):
    """
    Apply language detection and cleaning for a single row.
    Translation, PII masking, keywords handled in batch later.
    """
    subject = row.get("subject", "")
    body = row.get("body", "")

    lang = prep.detect_language(body)
    subject_clean = prep.clean_text(subject)
    body_clean = prep.clean_text(body)

    return {
        "ticket_id": row["ticket_id"],
        "lang": lang,
        "subject_clean": subject_clean,
        "body_clean": body_clean
    }




In [ ]:
# Apply row-wise detect language and cleaning
tickets_cleaned = tickets_df.progress_apply(preprocess_ticket_row, axis=1)
tickets_cleaned = pd.DataFrame(list(tickets_cleaned))
tickets_cleaned.head()

In [ ]:
# Initialize columns for translated text
tickets_cleaned["subject_translated"] = ""
tickets_cleaned["body_translated"] = ""

# Batch translate by language 

# Only languages with translation models will be processed here
for lang in prep.TRANSLATION_MODELS.keys():
    # Get indices of rows with this language
    idx = tickets_cleaned[tickets_cleaned["lang"] == lang].index
    if len(idx) == 0:
        continue  # skip if no rows in this language

    # Prepare text batches
    subjects = tickets_cleaned.loc[idx, "subject_clean"].tolist()
    bodies = tickets_cleaned.loc[idx, "body_clean"].tolist()

    # Translate in batch using preprocessing.py functions
    tickets_cleaned.loc[idx, "subject_translated"] = prep.translate_texts_batch(subjects, lang)
    tickets_cleaned.loc[idx, "body_translated"] = prep.translate_texts_batch(bodies, lang)

    # Log batch info
    print(f"[INFO] Translated batch of {len(idx)} tickets for language '{lang}'")

# Fill English/unknown rows with original cleaned content 
mask = tickets_cleaned["subject_translated"] == ""
tickets_cleaned.loc[mask, "subject_translated"] = tickets_cleaned.loc[mask, "subject_clean"]
tickets_cleaned.loc[mask, "body_translated"] = tickets_cleaned.loc[mask, "body_clean"]

# Verify
tickets_cleaned.head()

In [ ]:
# PII masking
tickets_cleaned["subject_masked"] = tickets_cleaned["subject_translated"].progress_apply(prep.mask_pii_en)
tickets_cleaned["body_masked"] = tickets_cleaned["body_translated"].progress_apply(prep.mask_pii_en)

# Verify
tickets_cleaned.head()


In [ ]:
tickets_cleaned.head(20)

In [ ]:
tickets_cleaned.to_csv("tickets_cleaned.csv", index=False)

In [ ]:

# Load from CSV into DataFrame
tickets_cleaned = pd.read_csv("tickets_cleaned.csv")

# Preview
tickets_cleaned.head()


In [ ]:
# Combine text columns into one for keyword extraction and later use
tickets_cleaned["combined_text"] = tickets_cleaned["subject_masked"] + "|| " + tickets_cleaned["body_masked"]

# Verify
tickets_cleaned.head()


In [ ]:
# Extract keywords from the combined text
keywords_list = prep.extract_keywords_batch(tickets_cleaned["combined_text"].tolist(), top_n=10)

# Store keywords as a comma-separated string
tickets_cleaned["keywords"] = [", ".join(kws) for kws in keywords_list]

# Verify
tickets_cleaned.head()


In [ ]:
# # Hello, Hi, I'm Sam Heikkinen. My card keeps getting errors when I try to pay. Balance should be fine, so I'm confused. I tried three times today, same result. Balance shkouldj be fine, so I'm confused.
# # Hallå, Hi, I'm Henrik Svensson. Jag har problem med card lost/stolen. Som jag nämnde tidigare, detta blockerar oss. Phone: +46 101 389 4079 IBAN: FI9106024211679678 Card: 4000 1234 5678 9010 Säg till om ni behöver skärmdumpar. Med vänlig hälsning.
# # Hei, Hi, I'm Sanna Mäkinen. Ongelma liittyy aiheeseen: API Rate Limit/Auth. 💳 IBAN: FI8663695284994003 As mentioned earlier, this is blocking our checkout. Ilmoittakaa jos tarvitsette kuvakaappauksia. Saldon pitäisi riittää, joten olen hämmentynyt. Ilmoittakaa jos tarvitsette kuvakaappauksia. Saldon pitäisi riittää, joten olen hämmentynyt.
# #Hej, Hi, I'm Matilda Virtanen. Jag har problem med dispute/chargeback. Testade också på en annan enhet. 🙏 Phone: +358 167 297 3630 Email: matilda.virtanen@example.com Card: 4970 **** **** 8430 Säg till om ni behöver skärmdumpar. Försökte tre gånger idag, samma resultat. Försökte tre gånger idag, samma resultat. Täckningen borde räcka, så jag är förvirrad. Tack!

# masked = preprocess_ticket("Hej, Hi, I'm Matilda Virtanen. Jag har problem med dispute/chargeback. Testade också på en annan enhet. 🙏 Phone: +358 167 297 3630 Email: matilda.virtanen@example.com Card: 4970 **** **** 8430 Säg till om ni behöver skärmdumpar. Försökte tre gånger idag, samma resultat. Försökte tre gånger idag, samma resultat. Täckningen borde räcka, så jag är förvirrad. Tack!")

In [ ]:
tickets_cleaned.head()

In [ ]:
# Combine text columns into one for keyword extraction and later use
tickets_cleaned["combined_text"] = tickets_cleaned["subject_translated"] + "|| " + tickets_cleaned["body_masked"]

# Verify
tickets_cleaned.head(10)

In [ ]:

import re

# Define greetings to remove (case-insensitive)
greetings_pattern = r'\b(hi|hello|hey)\b'

# Remove greetings from combined_text
tickets_cleaned["combined_text"] = tickets_cleaned["combined_text"].apply(
    lambda x: re.sub(greetings_pattern, '', x, flags=re.IGNORECASE)
)

# Remove extra whitespace after removal
tickets_cleaned["combined_text"] = tickets_cleaned["combined_text"].str.replace(r'\s+', ' ', regex=True).str.strip()

# Verify
tickets_cleaned.head(10)


In [ ]:
# Extract keywords from the combined text
keywords_list = prep.extract_keywords_batch(tickets_cleaned["combined_text"].tolist(), top_n=10)

# Store keywords as a comma-separated string
tickets_cleaned["keywords"] = [", ".join(kws) for kws in keywords_list]

# Verify
tickets_cleaned.head()

In [ ]:

max_words = tickets_cleaned["combined_text"].apply(lambda x: len(str(x).split())).max()
print("Maximum word count:", max_words)


In [ ]:
tickets_cleaned.to_csv("tickets_cleaned.csv", index=False)

In [ ]:
# Load from CSV into DataFrame
tickets_cleaned = pd.read_csv("tickets_cleaned.csv")

# Preview
tickets_cleaned.head(10)

In [ ]:
# Write DataFrame to Postgres
tickets_cleaned.to_sql(
    name="ticket_preprocessed",   
    con=engine,
    if_exists="replace",           
    index=False,                   
    dtype=None                     
)

print("Data successfully written to 'ticket_preprocessed' table.")